In [17]:
import text2term
import pandas
import pickle
import os
import pandas as pd
import json

In [2]:
text2term.cache_ontology("https://purl.obolibrary.org/obo/ncbitaxon.owl", "ncbitaxon")

2023-11-11 08:07:54 INFO [text2term.term_collector]: Loading ontology https://purl.obolibrary.org/obo/ncbitaxon.owl...
2023-11-11 08:21:48 INFO [text2term.term_collector]: ...done (ontology loading time: 589.27s)
2023-11-11 08:21:48 INFO [text2term.term_collector]: Collecting ontology term details...
2023-11-11 08:28:29 INFO [text2term.term_collector]: ...done: collected 2523789 ontology terms (collection time: 401.07s)
2023-11-11 08:33:36 INFO [text2term.t2t]: Filtered ontology terms to those of type: any
2023-11-11 08:33:37 INFO [text2term.t2t]: Caching ontology https://purl.obolibrary.org/obo/ncbitaxon.owl to: cache\ncbitaxon


In [8]:
with open(os.path.join('data','raw_species_results.pickle'),'rb') as infile:
    raw_species = pickle.load(infile)

print(raw_species.head(n=2))

             _id       _ignored  _score  \
0  GEO_GSE110840  [all.keyword]     1.0   
1  GEO_GSE110842  [all.keyword]     1.0   

                                                name  \
0  Sequencing of Caenorhabditis elegans deletion ...   
1  Sequencing of Caenorhabditis elegans overexpre...   

                                species  
0  [{'name': 'Caenorhabditis elegans'}]  
1  [{'name': 'Caenorhabditis elegans'}]  


In [15]:
print(len(raw_species))

raw_boom = raw_species.explode('species')
print(len(raw_boom))
print(raw_boom.head(n=2))

200837
202257
             _id       _ignored  _score  \
0  GEO_GSE110840  [all.keyword]     1.0   
1  GEO_GSE110842  [all.keyword]     1.0   

                                                name  \
0  Sequencing of Caenorhabditis elegans deletion ...   
1  Sequencing of Caenorhabditis elegans overexpre...   

                              species  
0  {'name': 'Caenorhabditis elegans'}  
1  {'name': 'Caenorhabditis elegans'}  


In [24]:
def pop_species(species_dict):
    if isinstance(species_dict,dict):
        x = species_dict['name']
    elif isinstance(species_dict,str):
        x = species_dict
    return x

## Raise a flag if the species was not formatted as a dictionary
def flag_species(species_dict):
    if isinstance(species_dict,dict):
        x = False
    elif isinstance(species_dict,str):
        x = True
    return x    

raw_boom['species_name'] = raw_boom.apply(lambda row: pop_species(row['species']),axis=1)
raw_boom['flag_raised'] = raw_boom.apply(lambda row: flag_species(row['species']),axis=1)
print(raw_boom.head(n=2))
clean_dict = raw_boom[['_id','name','species_name','flag_raised']].copy()

             _id       _ignored  _score  \
0  GEO_GSE110840  [all.keyword]     1.0   
1  GEO_GSE110842  [all.keyword]     1.0   

                                                name  \
0  Sequencing of Caenorhabditis elegans deletion ...   
1  Sequencing of Caenorhabditis elegans overexpre...   

                              species            species_name  flag_raised  
0  {'name': 'Caenorhabditis elegans'}  Caenorhabditis elegans        False  
1  {'name': 'Caenorhabditis elegans'}  Caenorhabditis elegans        False  


In [25]:
print(clean_dict.head(n=2))

             _id                                               name  \
0  GEO_GSE110840  Sequencing of Caenorhabditis elegans deletion ...   
1  GEO_GSE110842  Sequencing of Caenorhabditis elegans overexpre...   

             species_name  flag_raised  
0  Caenorhabditis elegans        False  
1  Caenorhabditis elegans        False  


In [27]:
no_dups = clean_dict['species_name'].unique().tolist()
print(len(no_dups))

2565


In [28]:
%%time
t2t_result = text2term.map_terms(no_dups, "ncbitaxon", use_cache=True)

2023-11-13 10:38:58 INFO [text2term.t2t]: Loading cached ontology from: cache\ncbitaxon\ncbitaxon-term-details.pickle
2023-11-13 10:41:30 INFO [text2term.t2t]: Filtered ontology terms to those of type: class
2023-11-13 10:41:30 INFO [text2term.t2t]: Mapping 2565 source terms to ncbitaxon
2023-11-13 10:46:52 INFO [text2term.t2t]: ...done (mapping time: 321.58s seconds)
CPU times: total: 7min 57s
Wall time: 8min 33s


In [31]:
t2t_result.sort_values(['Source Term','Mapping Score'], ascending = [True,False], inplace=True)
print(t2t_result.head(n=2))

                                  Source Term ID    Source Term  \
3180  http://ccb.hms.harvard.edu/t2t/R3Go4L6RwRz  Abies pinsapo   
3181  http://ccb.hms.harvard.edu/t2t/R3Go4L6RwRz  Abies pinsapo   

               Mapped Term Label Mapped Term CURIE  \
3180               Abies pinsapo   NCBITAXON:56046   
3181  Abies pinsapo var. pinsapo  NCBITAXON:928732   

                                      Mapped Term IRI  Mapping Score  Tags  
3180   http://purl.obolibrary.org/obo/NCBITaxon_56046          0.997  None  
3181  http://purl.obolibrary.org/obo/NCBITaxon_928732          0.923  None  


In [32]:
top_score = t2t_result.drop_duplicates('Source Term',keep='first')
print(len(top_score))

2564


In [34]:
t2t_dictdf = top_score[['Source Term','Mapped Term Label','Mapped Term CURIE', 'Mapping Score']].copy()
t2t_dictdf.rename(columns={'Source Term':'species_name', 'Mapped Term CURIE':'CURIE','Mapped Term Label':'Label'}, inplace=True)

In [36]:
mapped_dict = clean_dict.merge(t2t_dictdf, on='species_name', how='inner')
print(len(mapped_dict))
mapped_dict.drop_duplicates(keep='first', inplace=True)
print(len(mapped_dict))

202256
202182


In [41]:
no_flags = mapped_dict.loc[mapped_dict['flag_raised']==False]
flagged = mapped_dict.loc[mapped_dict['flag_raised']==True]
print(len(no_flags))

197509


In [43]:
no_flags.to_csv(os.path.join('data','t2t_mapped_no_flags.tsv'), sep='\t', header=True)
flagged.to_csv(os.path.join('data','t2t_mapped_flagged.tsv'), sep='\t', header=True)